In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

In [2]:
data=pd.read_csv(r'C:\Users\HP\Downloads\gld_price_data (2).csv')

In [3]:
X=data.drop(['Date','GLD'],axis=1)
Y=data['GLD']
print(X)
print(Y)

X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.2,random_state=2)

              SPX        USO      SLV   EUR/USD
0     1447.160034  78.470001  15.1800  1.471692
1     1447.160034  78.370003  15.2850  1.474491
2     1411.630005  77.309998  15.1670  1.475492
3     1416.180054  75.500000  15.0530  1.468299
4     1390.189941  76.059998  15.5900  1.557099
...           ...        ...      ...       ...
2285  2671.919922  14.060000  15.5100  1.186789
2286  2697.790039  14.370000  15.5300  1.184722
2287  2723.070068  14.410000  15.7400  1.191753
2288  2730.129883  14.380000  15.5600  1.193118
2289  2725.780029  14.405800  15.4542  1.182033

[2290 rows x 4 columns]
0        84.860001
1        85.570000
2        85.129997
3        84.769997
4        86.779999
           ...    
2285    124.589996
2286    124.330002
2287    125.180000
2288    124.489998
2289    122.543800
Name: GLD, Length: 2290, dtype: float64


In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
import seaborn as sns
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import matplotlib.pyplot as plt

# Ensure no inf values in the data
data.replace([np.inf, -np.inf], np.nan, inplace=True)



# Assuming X_train, y_train, X_test are defined
train_lstm(X_train, Y_train, X_test)



In [20]:
from pandas import concat
def train_xgboost(X_train, y_train, X_test, y_test):
    def series_to_supervised(data_input, n_in=1, n_out=1, dropnan=True):
        df= data_input["GLD"]
    
        n_vars = 1 if type(data_input) is list else data.shape[1]
        df =data_input["GLD"]
        cols = list()
        
        for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
    
        for i in range(0, n_out):
            cols.append(df.shift(-i))
     
        agg = concat(cols, axis=1)
     
        if dropnan:
            agg.dropna(inplace=True)
            return agg.values



    train = series_to_supervised(data, n_in=73)
    train



    from xgboost import XGBRegressor
    X_train, Y_train = train[0:2200, :-1], train[0:2200, -1]
    X_test, Y_test= train[2200:-6, :-1], train[2200:-6, -1]
    model = XGBRegressor(objective='reg:squarederror', n_estimators=10000, max_depth =9)
    model.fit(X_train, Y_train)
    yhat = model.predict(X_test)

    print("XGBoost Predictions:", yhat[:len(X_test)])
    print("Length of XGBoost Predictions:", len(yhat))


    return yhat

    r2=r2_score(Y_test,yhat)
    print(r2)
    
train_xgboost(X_train, Y_train, X_test, Y_test)

XGBoost Predictions: [127.744835 127.38933  127.0048   126.728325 125.67845  125.54693
 124.80582  124.46909  124.26511  123.99303  124.43893 ]
Length of XGBoost Predictions: 11


array([127.744835, 127.38933 , 127.0048  , 126.728325, 125.67845 ,
       125.54693 , 124.80582 , 124.46909 , 124.26511 , 123.99303 ,
       124.43893 ], dtype=float32)

In [21]:
#print("XGBoost Predictions:", yhat[:len(X_test)])
#print("Length of XGBoost Predictions:", len(yhat))

In [22]:
def random_forest(X_train, X_test, y_train, y_test):
    regressor = RandomForestRegressor(n_estimators=100)
regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train,Y_train)
test_data_prediction = regressor.predict(X_test)
Y_test=list(Y_test)
r2=r2_score(Y_test,test_data_prediction)
print(r2)

0.9888656648676653


In [25]:


# Predictions
#lstm_predictions = train_lstm(X_train, Y_train, X_test)
xgb_predictions = train_xgboost(X_train, Y_train, X_test, Y_test)
rf_predictions = random_forest(X_train, X_test, Y_train, Y_test)


#print(len(lstm_predictions))
print(len(xgb_predictions))
print(len(rf_predictions))


# Plotting
plt.figure(figsize=(15, 8))
sns.set_style('whitegrid')

# Ensure the lengths are consistent
plot_length = min(len(lstm_predictions), len(xgb_predictions), len(rf_predictions), len(Y_test))

plt.plot(range(plot_length), Y_test[:plot_length], label='Actual GLD Prices', color='black')
#plt.plot(range(plot_length), lstm_predictions[:plot_length], label='LSTM Predictions', color='blue')
plt.plot(range(plot_length), xgb_predictions[:plot_length], label='XGBoost Predictions', color='red')
plt.plot(range(plot_length), rf_predictions[:plot_length], label='Random Forest Predictions', color='green')

plt.xlabel('Time')
plt.ylabel('GLD Price')
plt.title('Comparison of LSTM, XGBoost, and Random Forest Predictions')
plt.legend()
plt.show()

XGBoost Predictions: [127.744835 127.38933  127.0048   126.728325 125.67845  125.54693
 124.80582  124.46909  124.26511  123.99303  124.43893 ]
Length of XGBoost Predictions: 11
11


TypeError: object of type 'NoneType' has no len()

In [24]:
plt.plot(dates_test, y_test, label='Actual', color='black')
plt.plot(dates_test, xgb_pred, label=f'XGBoost (Lag {int(best_xgboost["Lags"])})', linestyle='--')
#plt.plot(dates_test, lstm_pred, label=f'LSTM (Lag {int(best_lstm["Lags"])})', linestyle='--')
plt.plot(dates_test, gru_pred, label=f'GRU (Lag {int(best_gru["Lags"])})', linestyle='--')
plt.plot(dates_test, neuralprophet_pred, label=f'NeuralProphet (Lag {int(best_neuralprophet["Lags"])})', linestyle='--')

NameError: name 'dates_test' is not defined